In [356]:
import numpy as np
from qiskit_aer import Aer
from qiskit.circuit.library import QFT, PhaseEstimation, UnitaryGate
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit, QuantumRegister, AncillaRegister, ClassicalRegister
from scipy.linalg import expm
from qiskit.circuit.library.arithmetic.exact_reciprocal import ExactReciprocal
from qiskit.visualization import plot_histogram
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import scipy.linalg

from math import isclose
from qiskit.circuit.library.generalized_gates import UCRYGate



class HHL():

    def __init__(self,epsilon: float = 1e-2) -> None:
        
        super().__init__()

        self._epsilon = epsilon
        # Tolerance for the different parts of the algorithm as per [1]
        self._epsilon_r = epsilon / 3  # conditioned rotation
        self._epsilon_s = epsilon / 3  # state preparation
        self._epsilon_a = epsilon / 6  # hamiltonian simulation
        self.b = None
        self.b_norm=None

        self._scaling = None  # scaling of the solution

        self._sampler = None
        self.num_shots=10000
        # For now the default reciprocal implementation is exact
        self._exact_reciprocal = True
        # Set the default scaling to 1
        self.scaling = 1
        self.observable = None
        self.x = None


    def get_observable(self,qc):
        
        simulator = AerSimulator()
        circ = transpile(qc, simulator)

        # Run and get counts
        result = simulator.run(circ, shots=self.num_shots).result()
        counts = result.get_counts(circ)
        filtered_counts = {}
        for k, v in counts.items():
            # Split the result into data_qubits and flag_qubit
            data_qubits, flag_qubit = k.split(' ')
            # Check if the flag qubit is '1'
            if flag_qubit == '1':
                # Only keep the data qubits part in the final output
                filtered_counts[data_qubits] = v

        print(filtered_counts)

        self.observable = filtered_counts


    def postprocess(self):

        vector = np.zeros(len(self.b))
        tot_counts=0

        for index, count in self.observable.items():
            tot_counts=+count
        
        for index, count in self.observable.items():
            vector[int(index,2)] = count/tot_counts

        self.x = np.sqrt(vector*self.b_norm**2)*self.scaling
        print("quantum solution ",self.x)


    def construct_circuit(
        self,
        matrix: np.ndarray,
        vector: np.ndarray
    ) -> QuantumCircuit:

        # State preparation circuit - default is qiskit
        nb = int(np.log2(len(vector)))
        vector_circuit = QuantumCircuit(nb)
        self.b_norm = np.linalg.norm(vector)
        vector_circuit.initialize(vector / self.b_norm, list(range(nb)), None)

        # If state preparation is probabilistic the number of qubit flags should increase
        nf = 1

        # Hamiltonian simulation circuit
        matrix = np.array(matrix)  # Ensure the matrix has a compatible type
        #matrix_circuit = NumPyMatrix(matrix, evolution_time=2 * np.pi)

        #---------------------------------------------------------------------------------------------------------------

        # check if the matrix can calculate the condition number and store the upper bound
        kappa = np.linalg.cond(matrix)

        # Update the number of qubits required to represent the eigenvalues
        # The +neg_vals is to register negative eigenvalues because
        # e^{-2 \pi i \lambda} = e^{2 \pi i (1 - \lambda)}
        #nl = max(nb + 1, int(np.ceil(np.log2(kappa + 1)))) #+ neg_vals
        nl=2

        #--------------------------------------------------------------------------------------------------------------

        # check if the matrix can calculate bounds for the eigenvalues
        lambda_min, lambda_max = min(np.abs(np.linalg.eigvals(matrix))), max(np.abs(np.linalg.eigvals(matrix)))
        matrix = matrix/(lambda_max)
        U = expm(1j * matrix * 2 * np.pi)  # Unitary matrix corresponding to e^(-i*A*2π)

        U_circuit = UnitaryGate(U)

        # Constant so that the minimum eigenvalue is represented exactly, since it contributes
        # the most to the solution of the system. -1 to take into account the sign qubit
        #delta = self._get_delta(nl, lambda_min, lambda_max)
        # Update the scaling of the solution
        self.scaling = lambda_min

    
        #-----------------------------------------------------------------------------------------------------------------

        reciprocal_circuit = ExactReciprocal(nl, lambda_min)

        #-------------------------------------------------------------------------------------------------------------------
        
        # Initialise the quantum registers
        qb = QuantumRegister(nb)  # right hand side and solution
        ql = QuantumRegister(nl)  # eigenvalue evaluation qubits
        qf = QuantumRegister(nf)  # flag qubits
        
        cl = ClassicalRegister(nb)
        cf = ClassicalRegister(nf)


        qc = QuantumCircuit(qb, ql, qf,cl,cf)

        # State preparation
        qc.append(vector_circuit, qb[:])
        
        phase_estimation = PhaseEstimation(nl, U_circuit)
        
        # QPE
        qc.append(phase_estimation, ql[:] + qb[:])
        
        # Conditioned rotation
        #qc.append(reciprocal_circuit, ql[::-1] + [qf[0]]) #passing the ql reversed,  so doing the swaps
        angles = [0.0]
        Nl = 2 ** (nl) 

        # Angles to rotate by scaling / x, where x = i / nl
        for i in range(1, Nl):
            if isclose(lambda_min * Nl / i, 1, abs_tol=1e-5):
                angles.append(np.pi)
            elif lambda_min * Nl / i < 1:
                angles.append(2 * np.arcsin(lambda_min * Nl / i))
            else:
                angles.append(0.0)


        for i in range(0,len(angles)//nl):
            for angle, target in zip(angles[i*nl:], ql[::-1]):  # Reverse ql to match order
                print(angle)
                qc.cry(angle, target, qf[0])
        
        # QPE inverse
        qc.append(phase_estimation.inverse(), ql[:] + qb[:])

        qc.measure(qf,cf)

        qc.measure(qb,cl) 
    
        #------------------------------------------------------------------------------------------------

        self.get_observable(qc)
        self.postprocess()

        return qc

    def solve( self,matrix: np.ndarray,vector:  np.ndarray):
        
        self.b = vector
        solution = HHL()
        solution.state = self.construct_circuit(matrix, vector)

        return solution

In [357]:
# Example usage
A = [[1,0],
     [0,1/8]]


b = [1, 1]

hhl = HHL()
solution = hhl.solve(A, b)
#print("quantum solution ",solution.x)
print("classical solution ",scipy.linalg.solve(A,b))


print(solution.state)

0.0
1.0471975511965979
0.5053605102841573
0.33489615843937864
{'0': 4472, '1': 531}
quantum solution  [0.51301333 0.1767767 ]
classical solution  [1. 8.]
        ┌───────────────┐┌──────┐                                   »
  q502: ┤ circuit-16745 ├┤2     ├───────────────────────────────────»
        └───────────────┘│      │                                   »
q503_0: ─────────────────┤0 QPE ├──────────────■────────────────────»
                         │      │              │                    »
q503_1: ─────────────────┤1     ├────■─────────┼────────────■───────»
                         └──────┘┌───┴───┐┌────┴────┐┌──────┴──────┐»
  q504: ─────────────────────────┤ Ry(0) ├┤ Ry(π/3) ├┤ Ry(0.50536) ├»
                                 └───────┘└─────────┘└─────────────┘»
c247: 1/════════════════════════════════════════════════════════════»
                                                                    »
c248: 1/════════════════════════════════════════════════════════════»
      